# ModIty -  <font color='red'>Mod</font>el for the  <font color='red'>I</font>dentification of <font color='red'>Ty</font>peregions

In [ ]:
import pandas as pd
import numpy as np
import modity as mod

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### Klasse instanziieren

In [ ]:
bachelor = mod.Project(input_folder='Input')

### Datensatz auswählen und bearbeiten

In [ ]:
file = pd.read_csv(bachelor.input_path +'/AGS_AIRE.csv', sep=';', decimal=',', thousands='.',index_col=0)
file.drop(columns=['GEN','BEZ'], inplace=True)

### an Klasse übergeben

In [ ]:
bachelor.import_data_file(file, 'AGS')

### Normalverteilung überprüfen

In [ ]:
from scipy.stats import shapiro
import matplotlib.pyplot as plt
def shapiro_test(data):
    shapiro_p_values = []
    for i in range(data.shape[1]):
        shapiro_p_values.append(shapiro(data[:,i])[1])

    plt.bar(range(data.shape[1]),shapiro_p_values)
    plt.locator_params(axis='x', nbins=data.shape[1]*2)
    plt.grid()
    plt.title('Shapiro - Test')
    plt.show()

    #shapiro_p_values>np.mean(shapiro_p_values)
    print('Anzahl der Normalverteilten')
    print(sum(np.ravel(shapiro_p_values)>0.5))
    return shapiro_p_values

In [ ]:
p_values_shapiro = shapiro_test(bachelor.df_data.values)

In [ ]:
bachelor.correlation(method='kendall',heatmap=True, memory_set=0, save=False)

### Variante 2

In [ ]:
bachelor.select_features(reject=[0,1,3,4,5,6,8,10,13,14,15,16,17], memory_set=0)

In [ ]:
bachelor.correlation(method='kendall',heatmap=True,threshold=0.8, memory_set=1, save=False)

### Verteilung der Features

In [ ]:
bachelor.violinplots(memory_set=1, save=False)

### es werden keine Ausreißer identifiziert sondern gleich nach der optimalen Cluster anzahl gesucht

In [ ]:
bachelor.optimal_k(memory_set=1, kmax=20, scaler='standard', plot=True)

- stark variierende Empfehlungen
- Sil k=8
- DB k=19
- CH k=18
- SSR k=7
- Auffällig hoher Wert bei k=2 für Sil 

### k=2 deutet auf Ausreißer hin, die den Clusterindizes und Kmeans stören

In [ ]:
bachelor.loop_clustering(memory_set=1, k_cluster=2, max_loops=30)

- sehr robuste Zuteilung

In [ ]:
size_var2_m1_k2 = bachelor.cluster_sizes(memory_set=1, k_cluster=2)

- kleineres Cluster wird als Ausreißercluster entfernt

### Sample identifizieren

In [ ]:
cluster_ident= (0,0)
labels_var2_m1 = bachelor.get_labels(memory_set=1, k_cluster=2, loop=cluster_ident[0])
single_var2_m1 = labels_var2_m1[~labels_var2_m1.ne(cluster_ident[1])]
single_var2_m1 = bachelor.df_transfer.Name[single_var2_m1.index]
#single_var2_m1.to_latex('./graphics/outlier_var2_m1.tex')
single_var2_m1

### samples entfernen

In [ ]:
bachelor.select_samples(memory_set=1, reject=single_var2_m1.index)

## erneut optimieren

In [ ]:
bachelor.optimal_k(memory_set=2, kmax=20, scaler='standard', plot=True)

- eindeutigere Ergebnisse werden erzielt
- k=6 hat gute Werte

In [ ]:
bachelor.loop_clustering(memory_set=2, k_cluster=6, max_loops=30)

In [ ]:
size_var2_m2_k6 = bachelor.cluster_sizes(memory_set=2, k_cluster=6)

- kleines cluster mit nur 4 samples kann identifiziert werden

### Sample identifizieren

In [ ]:
cluster_ident= (0,2)
labels_var2_m2_k6 = bachelor.get_labels(memory_set=2, k_cluster=6, loop=cluster_ident[0])
single_var2_m2_k6 = labels_var2_m2_k6[~labels_var2_m2_k6.ne(cluster_ident[1])]
single_var2_m2_k6 = bachelor.df_transfer.Name[single_var2_m2_k6.index]
#single_var2_m2_k6.to_latex('./graphics/outlier_var2_m2_k6_cluster4.tex')
single_var2_m2_k6

### sampes entfernen

In [ ]:
bachelor.select_samples(memory_set=2, reject=single_var2_m2_k6.index)

## neu optimieren

In [ ]:
bachelor.optimal_k(memory_set=3, kmax=20, scaler='standard', plot=True)

-k = 6 als optimaler Wert

In [ ]:
bachelor.loop_clustering(memory_set=3, k_cluster=6, max_loops=30)

In [ ]:
size_var2_m3_k6 = bachelor.cluster_sizes(memory_set=3, k_cluster=6)

### springer identifizeren

In [ ]:
borderliner_var2_m3_k6 = bachelor.borderliner(memory_set=3, k_cluster=6)

### borderliner entfernen

In [ ]:
bachelor.select_samples(memory_set=3, reject=borderliner_var2_m3_k6.index)

## neu optimieren

In [ ]:
bachelor.optimal_k(memory_set=4, kmax=20, scaler='standard', plot=True)

- k=6
- werte verbessern sich leicht

In [ ]:
bachelor.loop_clustering(memory_set=4, k_cluster=6, max_loops=30)

In [ ]:
size_var2_m4_k6 = bachelor.cluster_sizes(memory_set=4, k_cluster=6)

# eindeutiges ergebnis

### mapping der clusterzuweisung an variante 1 anpassen
- manuell clusterzuweisung abgleichen
- dann speicher manipulieren

In [ ]:
bachelor.plot_silhouettes(memory_set=4, k_cluster=6, loop=0)

In [ ]:
bachelor.plot_cluster(memory_set=4, k_cluster=6, loop=0, save=False)

In [ ]:
size_var2_m4_k6

### typregionen identifizieren

In [ ]:
typregions= bachelor.ident_typregions(memory_set=4, k_cluster=6, loop=0)
#typregions.join(size).to_latex('./graphics/typregions_var2.tex')
typregions.join(size_var2_m4_k6[0])

### typregionen visualisieren

In [ ]:
bachelor.plot_typregions(memory_set=4, k_cluster=6, loop=0,save=False)

# Medianwerte der Cluster berechnen

In [ ]:
clustering = bachelor.get_clustering(memory_set=4, k_cluster=6, loop=0)
clustering.Cluster = clustering.Cluster.astype(int)
clustering
output = clustering.groupby('Cluster').median().join(size_var2_m4_k6[0].rename('Clustergröße'))
#output.iloc[:,[6,0,1,2,3,4,5]].T.to_latex('./graphics/median_var2.tex')
#output.T.to_latex('./graphics/median.tex')
output.iloc[:,[6,0,1,2,3,4,5]].T

# zusätzliche Plots

In [ ]:
import matplotlib.cm as cm
cmap = cm.get_cmap("Spectral_r")
cmd = bachelor.cmap_discretize(cmap, N=7)
cm.register_cmap("var1_r", cmd)

In [ ]:
df_clustering = bachelor.get_clustering(memory_set=4, k_cluster=6, loop=0)
df_clustering.iloc[:,:-2] = bachelor.normalization(df_clustering.iloc[:,:-2], scaler='minmax')
df_clustering.Cluster = df_clustering.Cluster.astype(int)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# boxplots
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(25,20))
for n, feature in enumerate(df_clustering.columns[:-2]):
    plt.subplot(2,3, n+1 )
    #sns.violinplot(x ='Cluster',y=feature, data= df_clustering)
    sns.swarmplot(x ='Cluster',y=feature, data= df_clustering, color="black", edgecolor="gray")
    sns.boxplot(x ='Cluster',y=feature, palette='var1_r', data= df_clustering)
    plt.xlabel('Cluster', fontsize=12)
    plt.ylabel(feature, fontsize=12)
    plt.title(feature, fontsize=15)
plt.show()
#fig.savefig('./graphics/boxplots_feature_var2.pdf',bbox_inches='tight')

# vergleich der Clusteranzahlen

In [ ]:
bachelor.plot_k_scores(memory_set=[2,3,4], save=False)

# Ende